# Лабораторная работа 7. Управление ресурсами в однопроцессорной системе с неоднородными заявками
Выполнил: Пакало Александр Сергеевич, студент РТ5-81Б

## Задание 1
В однопроцессорную систему случайным образом поступают на выполнение $m$
разных типов программ, отличающихся известной трудоемкостью
$Q_1, Q_2,\dots, Q_m$.
Входящий поток простейший с интенсивностью $\lambda$.

Представив данную систему как одноканальную СМО с неограниченной очередью,
вычислить среднее время обслуживания программ, считая длительность
обслуживания случайной величиной (теоретически и экспериментально).

Реализовать алгоритм SPT, выбирая из очереди заявки в соответствии с их
приоритетом по трудоемкости. Рассчитать среднее время обслуживания программ.
Сравнить полученные результаты.

Реализовать алгоритм RR при заданном кванте времени $q$. Оценить среднее время
обслуживания программ. Сравнить полученные результаты. Выяснить, как влияет
величина кванта на среднее время обслуживания программ.

In [32]:
Variant <- 5
set.seed(Variant)
m <- sample(c(6:20), 1)
lambda <- runif(1, 0.1, 2)
Q <- rexp(m, 0.3)
q <- sample(c(1:4), 1)
View(data.frame(m, q, lambda))
print(Q)

m,q,lambda
<int>,<int>,<dbl>
7,1,1.401915


[1] 0.2417931 1.3403831 0.1863999 2.0529013 0.2552007 2.9172096 6.6709868


Заведем таблицу результатов

In [33]:
results <- data.frame("-", "-", "-", "-")
colnames(results) <- c("M/M/1/infty theoretical", "M/M/1/infty practical", "SPT", "Round Robin")
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT,Round Robin
<chr>,<chr>,<chr>,<chr>
-,-,-,-


### СМО вида $М/М/1/\infty$
Представим данную систему как одноканальную СМО с неограниченной очередью.

#### Теоретически

In [34]:
t2 <- mean(Q)
mu <- 1 / t2
mu

[1] 0.5122623

In [35]:
y <- lambda / mu
y

[1] 2.736714

Так как $y > 1$, поменяем $\lambda$.

In [36]:
lambda <- 0.3

In [37]:
t2 <- mean(Q)
mu <- 1 / t2
mu

[1] 0.5122623

In [38]:
y <- lambda / mu
y

[1] 0.5856375

In [39]:
results[1] <- 1 / mu / (1 - y)
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT,Round Robin
<dbl>,<chr>,<chr>,<chr>
4.711152,-,-,-


#### Численно

In [40]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

env <- simmer("SuperDuperSim")
env

simmer environment: SuperDuperSim | now: 0 | next: 
{ Monitor: in memory }

In [41]:
programs <- trajectory("programs' path") %>%
    seize("server", amount = 1) %>%
    timeout(function() rexp(1, mu)) %>%
    release("server", amount = 1)

In [42]:
SIMULATION_TIME <- 10000

env %>%
    add_resource("server", 1) %>%
    add_generator("programs", programs, function() rexp(1, lambda)) %>%
    run(until = SIMULATION_TIME)

simmer environment: SuperDuperSim | now: 10000 | next: 10003.7234989748
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 0(1) | queue status: 0(Inf) }
{ Source: programs | monitored: 1 | n_generated: 3029 }

In [43]:
arrivals <- env %>%
    get_mon_arrivals()
arrivals

name,start_time,end_time,activity_time,finished,replication
<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<int>
programs0,1.282143,4.177776,2.89563311,TRUE,1
programs1,1.770443,4.258100,0.08032395,TRUE,1
programs2,4.408824,11.193970,6.78514566,TRUE,1
programs3,7.054637,13.693605,2.49963501,TRUE,1
programs4,11.334202,14.437538,0.74393268,TRUE,1
programs5,13.441375,18.684576,4.24703859,TRUE,1
programs6,19.294948,19.813446,0.51849776,TRUE,1
programs7,21.252343,21.511088,0.25874508,TRUE,1
programs8,25.357193,27.734487,2.37729349,TRUE,1


In [44]:
results[2] <- mean(arrivals %>% with(end_time - start_time))
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT,Round Robin
<dbl>,<dbl>,<chr>,<chr>
4.711152,4.906583,-,-


### Алгоритм SPT

In [45]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

spt.env <- simmer("SuperDuperSptSim")
spt.env

simmer environment: SuperDuperSptSim | now: 0 | next: 
{ Monitor: in memory }

Добавим $m$ генераторов. Каждый будет иметь приоритет в зависимости от
скорости выполнения программы. Генератор, создающий программы с наибольшей
длительностью выполнения, будет иметь наименьший приоритет.

In [46]:
SIMULATION_TIME <- 100000

spt.env %>%
    add_resource("server", 1)

Q_sorted_decr <- sort(Q, decreasing = TRUE)

spt.programs_trajectory <- function(time_to_execute) {
    return(
        trajectory("programs' path") %>%
            seize("server", 1) %>%
            timeout(time_to_execute) %>%
            release("server", 1)
    )
}

for (Q_i in seq_along(Q_sorted_decr)) {
    priority <- Q_i

    name <- paste0("programs", Q_i)

    spt.env %>% add_generator(
        name,
        spt.programs_trajectory(Q_sorted_decr[Q_i]),
        priority = priority,
        preemptible = priority,
        distribution = function() rexp(1, lambda / m)
    )
}

spt.env %>%
    run(until = SIMULATION_TIME)

simmer environment: SuperDuperSptSim | now: 0 | next: 
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 0(1) | queue status: 0(Inf) }

simmer environment: SuperDuperSptSim | now: 1e+05 | next: 100004.231080445
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 0(1) | queue status: 0(Inf) }
{ Source: programs1 | monitored: 1 | n_generated: 4270 }
{ Source: programs2 | monitored: 1 | n_generated: 4238 }
{ Source: programs3 | monitored: 1 | n_generated: 4265 }
{ Source: programs4 | monitored: 1 | n_generated: 4219 }
{ Source: programs5 | monitored: 1 | n_generated: 4199 }
{ Source: programs6 | monitored: 1 | n_generated: 4267 }
{ Source: programs7 | monitored: 1 | n_generated: 4282 }

In [47]:
spt.arrivals <- spt.env %>%
    get_mon_arrivals()
spt.arrivals

name,start_time,end_time,activity_time,finished,replication
<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<int>
programs10,3.139379,9.810366,6.6709868,TRUE,1
programs70,5.394727,9.996766,0.1863999,TRUE,1
programs11,7.332261,16.667752,6.6709868,TRUE,1
programs50,11.933127,16.922953,0.2552007,TRUE,1
programs40,12.175471,18.263336,1.3403831,TRUE,1
programs30,13.050160,20.316237,2.0529013,TRUE,1
programs41,19.239142,21.656620,1.3403831,TRUE,1
programs31,19.806402,23.709522,2.0529013,TRUE,1
programs20,22.848134,26.626731,2.9172096,TRUE,1


In [48]:
results[3] <- mean(spt.arrivals %>% with(end_time - start_time))
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT,Round Robin
<dbl>,<dbl>,<dbl>,<chr>
4.711152,4.906583,3.865118,-


### Алгоритм Round Robin
Реализуем Round Robin с помощью simmer.

In [54]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

rr.env <- simmer("SuperDuperRoundRobinSim")
rr.env

simmer environment: SuperDuperRoundRobinSim | now: 0 | next: 
{ Monitor: in memory }

In [55]:
rr.programs <- trajectory("rr.programs' path") %>%
    seize("server", 1) %>%
    timeout(function() rexp(1, mu)) %>%
    release("server", 1)

In [56]:
SIMULATION_TIME <- 10000

rr.env %>%
    add_resource("server", 1) %>%
    add_generator("rr.programs", rr.programs, function() rexp(1, lambda)) %>%
    run(until = SIMULATION_TIME)

simmer environment: SuperDuperRoundRobinSim | now: 10000 | next: 10001.4783564276
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 1(1) | queue status: 0(Inf) }
{ Source: rr.programs | monitored: 1 | n_generated: 2991 }

In [57]:
rr.arrivals <- rr.env %>%
    get_mon_arrivals()
rr.arrivals

name,start_time,end_time,activity_time,finished,replication
<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<int>
rr.programs0,2.675207,5.067564,2.3923566,TRUE,1
rr.programs1,3.339503,8.219849,3.1522857,TRUE,1
rr.programs2,5.307651,9.278114,1.0582642,TRUE,1
rr.programs3,9.254496,11.284527,2.0064131,TRUE,1
rr.programs4,12.325777,13.639013,1.3132359,TRUE,1
rr.programs5,14.726918,16.487386,1.7604677,TRUE,1
rr.programs6,15.826359,21.327240,4.8398539,TRUE,1
rr.programs7,19.541841,22.855573,1.5283329,TRUE,1
rr.programs8,25.985022,26.430180,0.4451574,TRUE,1


In [58]:
results[4] <- mean(rr.arrivals %>% with(end_time - start_time))
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT,Round Robin
<dbl>,<dbl>,<dbl>,<dbl>
4.711152,4.906583,3.865118,4.765589


Как видно, практические вычисления совпадают теоретическими с некоторой
погрешностью, которая уменьшается при увеличении числа экспериментов.

При этом система, выполненная с помощью алгоритма Round Robin оказалась
быстрее обычной системы, а система, реализованная с алгоритмом SPT - самой
быстрой.